In [7]:
%run ./setup-aml.ipynb
aml_login()

Tenant ID: 72f988bf-86f1-41af-91ab-2d7cd011db47
Subscription ID: 3e0e14b3-7e28-4da7-97de-0f5cb324f030
Resource Group:  ml
Workspace Name: ml-service
Library configuration succeeded


In [1]:
import os
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import EstimatorStep
from azureml.train.dnn import PyTorch

In [2]:
    num_epochs = PipelineParameter(name='num_epochs', default_value=25)
    batch_size = PipelineParameter(name='batch_size', default_value=16)
    learning_rate = PipelineParameter(name='learning_rate', default_value=0.001)
    momentum = PipelineParameter(name='momentum', default_value=0.9)

In [12]:
import os
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import EstimatorStep
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig
from azureml.pipeline.steps import CommandStep

def train_step(train_dir, valid_dir, compute_target, work_space):
    '''
    This step will fine-tune a RESNET-18 model on our dataset using PyTorch. 
    It will use the corresponding input image directories as training and validation data.

    :param train_dir: The reference to the directory containing the training data
    :type train_dir: DataReference
    :param valid_dir: The reference to the directory containing the validation data
    :type valid_dir: DataReference
    :param compute_target: The compute target to run the step on
    :type compute_target: ComputeTarget
    
    :return: The preprocess step, step outputs dictionary (keys: model_dir)
    :rtype: EstimatorStep, dict
    '''

    num_epochs = PipelineParameter(name='num_epochs', default_value=25)
    batch_size = PipelineParameter(name='batch_size', default_value=16)
    learning_rate = PipelineParameter(name='learning_rate', default_value=0.001)
    momentum = PipelineParameter(name='momentum', default_value=0.9)

    model_dir = PipelineData(
        name='model_dir', 
        pipeline_output_name='model_dir',
        datastore=train_dir.datastore,
        output_mode='mount',
        is_directory=True)

    outputs = [model_dir]
    outputs_map = { 'model_dir': model_dir }


    curated_env_name = 'AzureML-PyTorch-1.6-GPU'
    pytorch_env = Environment.get(workspace=work_space, name=curated_env_name)
    step = EstimatorStep(
        estimator=estimator,
        estimator_entry_script_arguments=[
            '--train_dir', train_dir, 
            '--valid_dir', valid_dir, 
            '--output_dir', model_dir, 
            '--num_epochs', num_epochs, 
            '--batch_size', batch_size,
            '--learning_rate', learning_rate, 
            '--momentum', momentum
        ],
        inputs=[train_dir, valid_dir],
        compute_target=compute_target,
        outputs=outputs,
        allow_reuse=False)
    train_config = ScriptRunConfig(source_directory=os.path.dirname(os.path.abspath(__file__)),
                      script='pytorch_train.py',
                      arguments=['--num_epochs', 30, '--output_dir', './outputs',  '--train_dir', train_dir, 
                                      '--valid_dir', valid_dir, 
                                      '--output_dir', model_dir, 
                                      '--num_epochs', num_epochs, 
                                      '--batch_size', batch_size,
                                      '--learning_rate', learning_rate, 
                                      '--momentum', momentum
                                    ],
                      compute_target=compute_target,
                      environment=pytorch_env)
    
    train_step = CommandStep(name='train-step', runconfig=train_config)
    
    return train_step, outputs_map

In [ ]:
train_config = ScriptRunConfig(source_directory=os.path.dirname(os.path.abspath(__file__),
    script='train.py',arguments=['--num_epochs', 30, '--output_dir', './outputs',  '--train_dir', train_dir, 
                                      '--valid_dir', valid_dir, 
                                      '--output_dir', model_dir, 
                                      '--num_epochs', num_epochs, 
                                      '--batch_size', batch_size,
                                      '--learning_rate', learning_rate, 
                                      '--momentum', momentum
                                    ],
                                    compute_target=compute_target,
                                    environment=pytorch_env)

In [15]:
from azureml.core import Workspace
workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-b66b1b52-c2d7-4600-8c2e-8051b69239cc",
  "account_name": "mlservice7009088601",
  "protocol": "https",
  "endpoint": "core.windows.net"
}